# Preprocessing for language_tool analysis
- Get list of misspellings, preprocessed (i. e. data_error_types)
- Get continuous text
    - Needs to be preprocessed

In [20]:
import pandas as pd
import sys 
import os
import csv
#print(sys.path)
#print(os.getcwd())

sys.path.insert(0, '../../Lisa')
import litkey_2

# Do not truncate columns of DataFrame
pd.set_option('display.max_rows', None)

## 1 - Get list of misspellings, preprocessed
i. e. data_error_types
- 'original' column only
- export to txt oder csv

In [2]:
data_error_types = litkey_2.load(litkey_data_path = "../../litkey-data/")

C:\Users\User\bodu-spell\spelling_correction\Lisa\Experiment_language_tool\../../Lisa\litkey_2.py:20: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  data = helpers_litkey_2.combineCsvsToDataframe(litkey_data_path)


In [3]:
data_error_types.head(10)

,original,corrected,filename,freq_ori,freq_cor,freq_tup
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73
2,dan,dann,01-005-2-III-Eis.csv,627,651,621
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19
8,fom,vom,01-005-2-III-Eis.csv,13,16,13
9,gawen,kaufen,01-006-2-III-Eis.csv,1,6,1


In [4]:
data_error_types_orig = data_error_types.original

In [5]:
data_error_types_orig.head(10)

0        belt
1        kukt
2         dan
3    gekricht
4       leken
5        felt
6       wolte
7        lekt
8         fom
9       gawen
Name: original, dtype: object

In [6]:
# Export
data_error_types_orig.to_csv("data_error_types_origs.txt", header=False, index=False)

## 2 - Get continuous text
- Exclude characters from normalisation
    - ( NA-cases (line ends(^)) )
    - ( headline markers (\h) )
    
    
    - "|" and "_", i. e. original is one/two-word whereas target is two/one-word
    - intended line-break characters '^', '-^', '^-'

In [2]:
# In data_raw_token, NA-cases and headline markers are already removed
data_raw_token = litkey_2.load_raw(litkey_data_path = "../../litkey-data/")

In [3]:
# Remove "|" and "_" characters, i. e. original is one/two-word whereas target is two/one-word
data_raw_token['original']=data_raw_token['original'].str.replace(r'[\|\_]', '', regex=True)
data_raw_token['corrected']=data_raw_token['corrected'].str.replace(r'[\|\_]', '', regex=True)
    
# Remove intended line-break characters '^', '-^', '^-'
pattern = r'(\-\^)|(\^\-)|(\^)'
data_raw_token['original'] = data_raw_token['original'].str.replace(pattern, '', regex=True)


# Remove words containing illegible character(s), i. e. containing '*' (original)
data_raw_token = data_raw_token[~data_raw_token.original.str.contains(r'\*')]
    
# Remove non-words ('?' target not identifiable or no standardized spelling | '~' target non existing word form)
data_raw_token = data_raw_token[~data_raw_token.corrected.str.contains(r'\?|\~')]

In [4]:
display(data_raw_token.head(10))
display(data_raw_token.tail(10))

,original,corrected,filename
0,Lea,Lea,01-005-2-III-Eis.csv
1,und,und,01-005-2-III-Eis.csv
2,Lars,Lars,01-005-2-III-Eis.csv
3,und,und,01-005-2-III-Eis.csv
4,Dodo,Dodo,01-005-2-III-Eis.csv
5,gehen,gehen,01-005-2-III-Eis.csv
6,Eis,Eis,01-005-2-III-Eis.csv
7,kaufen,kaufen,01-005-2-III-Eis.csv
8,.,.,01-005-2-III-Eis.csv
9,Dodo,Dodo,01-005-2-III-Eis.csv


,original,corrected,filename
212493,zu,zu,10-693-4-IV-Weg.csv
212494,schenken,schenken,10-693-4-IV-Weg.csv
212495,.,.,10-693-4-IV-Weg.csv
212496,P.S.,P.S.,10-693-4-IV-Weg.csv
212497,Lars,Lars,10-693-4-IV-Weg.csv
212498,ist,ist,10-693-4-IV-Weg.csv
212499,in,in,10-693-4-IV-Weg.csv
212500,Lea,Lea,10-693-4-IV-Weg.csv
212501,verknallt,verknallt,10-693-4-IV-Weg.csv
212502,.,.,10-693-4-IV-Weg.csv


In [5]:
# for each filename
storys = data_raw_token.filename.value_counts().index.to_list()
print(type(storys)) # Series
list(storys)
print(type(storys))
#type(storys[0]) # numpy int
len(storys)

<class 'list'>
<class 'list'>


1922

In [21]:
# Export filewise
import os
for s in storys:
    temp = data_raw_token[data_raw_token.filename==s]
    s_cut = s.split('.csv')[0].strip()
    temp.original.to_csv(os.path.join('Input_texts_csv/'+s_cut+'.csv'), sep='\n', quoting=csv.QUOTE_NONE, header=False, index=False, encoding='utf-8')
# Finish

In [12]:
storys[1].split('.csv')[0].strip()

'09-371-4-III-Seilbahn'

In [8]:
data_raw_token_orig = data_raw_token.original

In [9]:
data_raw_token_orig.head(20)

0         Lea
1         und
2        Lars
3         und
4        Dodo
5       gehen
6         Eis
7      kaufen
8           .
9        Dodo
10       belt
11        den
12    Eismann
13         an
14          .
15        der
16    Eismann
17       kukt
18       Dodo
19         an
Name: original, dtype: object

In [14]:
# Export
data_raw_token_orig.to_csv("./Input/data_raw_token_LTprep_origs.csv", sep='\n', quotechar="'" , header=False, index=False)